# Neptune + PyTorch

## Introduction

This guide will show you how to:

* Initialize Neptune and create a `run`,
* Log run configuration and metrics to Neptune,
* Log model hyperparameters, architecture, and weights to Neptune,
* Log torch tensors as images to Neptune.

## Before you start

This notebook example lets you try out Neptune as an anonymous user, with zero setup.

* If you are running the notebook on your local machine, you need to have [Python](https://www.python.org/downloads/) and [pip](https://pypi.org/project/pip/) installed.
* If you want to see the example recorded to your own workspace instead:
    * Create a Neptune account → [Take me to registration](https://neptune.ai/register)
    * Create a Neptune project that you will use for tracking metadata → [Tell me more about projects](https://docs.neptune.ai/administration/projects)

## Install Neptune and dependencies

In [ ]:
! pip install neptune-client numpy==1.19.5 torch==1.9.0 torchvision==0.10.0

## Basic example

**Import libraries**

In [ ]:
import neptune.new as neptune
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

from torchvision import datasets, transforms

### Create a Neptune *run*

To log metadata to the Neptune project, you need the `project name` and the `api_token`.

To make this example easy to follow, we have created a public project **'common/optuna-integration'** and a shared user **'neptuner'** with the API token **'ANONYMOUS'**. As you will see in the code cell below.

**(Optional)** To log to your Neptune project:

* [Create a Neptune account](https://app.neptune.ai/register/)

* [Find your API token](https://docs.neptune.ai/getting-started/installation#authentication-neptune-api-token)
* [Find your project name](https://docs.neptune.ai/getting-started/installation#setting-the-project-name)

Pass your credentials to project and api_token arguments of neptune.init()

`run = neptune.init(api_token="YOUR_API_TOKEN", project="YOUR_WORKSPACE/YOUR_PROJECT")` # pass your credentials


In [ ]:
run = neptune.init(
    project="common/pytorch-integration", tags="Colab Notebook", api_token="ANONYMOUS"
)

Running this cell creates a run in Neptune, and you can log model building metadata to it.

**Click on the link above to open the run in Neptune app.** For now, it is empty, but you should keep the tab open to see what happens next

### Log Hyperparameters

In [ ]:
parameters = {
    "lr": 1e-2,
    "bs": 128,
    "input_sz": 32 * 32 * 3,
    "n_classes": 10,
    "model_filename": "basemodel",
    "device": torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
}

In [ ]:
run["config/hyperparameters"] = parameters

### Log Config
Model and Dataset

In [ ]:
class BaseModel(nn.Module):
    def __init__(self, input_sz, hidden_dim, n_classes):
        super(BaseModel, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(input_sz, hidden_dim * 2),
            nn.ReLU(),
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Linear(hidden_dim // 2, n_classes),
        )

    def forward(self, input):
        x = input.view(-1, 32 * 32 * 3)
        return self.main(x)

In [ ]:
model = BaseModel(parameters["input_sz"], parameters["input_sz"], parameters["n_classes"]).to(
    parameters["device"]
)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=parameters["lr"])

Log model, criterion and optimizer name

In [ ]:
run["config/model"] = type(model).__name__
run["config/criterion"] = type(criterion).__name__
run["config/optimizer"] = type(optimizer).__name__

In [ ]:
data_dir = "data/CIFAR10"
compressed_ds = "./data/CIFAR10/cifar-10-python.tar.gz"
data_tfms = {
    "train": transforms.Compose(
        [
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ]
    ),
    "val": transforms.Compose(
        [
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ]
    ),
}

In [ ]:
trainset = datasets.CIFAR10(data_dir, transform=data_tfms["train"], download=True)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=parameters["bs"], shuffle=True, num_workers=2
)
validset = datasets.CIFAR10(data_dir, train=False, transform=data_tfms["train"], download=True)
validloader = torch.utils.data.DataLoader(validset, batch_size=parameters["bs"], num_workers=2)

dataset_size = {"train": len(trainset), "val": len(validset)}

### Log dataset details

In [ ]:
run["config/dataset/path"] = data_dir
run["config/dataset/transforms"] = data_tfms
run["config/dataset/size"] = dataset_size

### Log losses and metrics 
Training Loop

In [ ]:
for i, (x, y) in enumerate(trainloader, 0):
    x, y = x.to(parameters["device"]), y.to(parameters["device"])
    optimizer.zero_grad()
    outputs = model.forward(x)
    _, preds = torch.max(outputs, 1)
    loss = criterion(outputs, y)
    acc = (torch.sum(preds == y.data)) / len(x)

    run["training/batch/loss"].log(loss)

    run["training/batch/acc"].log(acc)

    loss.backward()
    optimizer.step()

## More Options

### Log model architecture and weights
You need to have saved these files to disk using a helper function like save_model before trying to upload to neptune 

In [ ]:
fname = parameters["model_filename"]

# Saving model architecture to .txt
with open(f"./{fname}_arch.txt", "w") as f:
    f.write(str(model))

# Saving model weights .pth
torch.save(model.state_dict(), f"./{fname}.pth")

In [ ]:
run[f"io_files/artifacts/{parameters['model_filename']}_arch"].upload(
    f"./{parameters['model_filename']}_arch.txt"
)
run[f"io_files/artifacts/{parameters['model_filename']}"].upload(
    f"./{parameters['model_filename']}.pth"
)

### Log images 

**Log Torch Tensors as images**  
You can log PyTorch Tensors (2d or 3d) directly from the memory, and have them visualized as an image. For more about logging torch tensor see [what you can log and display](https://docs.neptune.ai/you-should-know/what-can-you-log-and-display#pytorch-tensor)
and [field types](https://docs.neptune.ai/api-reference/field-types#fileseries) to understand how to upload multiple files.

Get predictions

In [ ]:
from neptune.new.types import File
import torch.nn.functional as F

classes = [
    "airplane",
    "automobile",
    "bird",
    "cat",
    "deer",
    "dog",
    "frog",
    "horse",
    "ship",
    "truck",
]
dataiter = iter(validloader)
images, labels = dataiter.next()
model.eval()

if torch.cuda.is_available():
    model.to("cpu")

n_samples = 50
img = images[:n_samples]
probs = F.softmax(model(img), dim=1)
probs = probs.data.numpy()

Decode probs and Log images tensors  
Log Series of Tensors as Image and Predictions. 

In [ ]:
for i, ps in enumerate(probs):
    pred = classes[np.argmax(ps)]
    gt = classes[labels[i]]
    description = "\n".join(
        ["class {}: {}%".format(classes[n], round(p * 100, 2)) for n, p in enumerate(ps)]
    )

    run["images/predictions"].log(
        File.as_image(img[i].squeeze().permute(2, 1, 0).clip(0, 1)),
        name=f"{i}_{pred}_{gt}",
        description=description,
    )

## Stop run

<font color=red>**Warning:**</font><br>
Once you are done logging, you should stop tracking the run using the `stop()` method.
This is needed only while logging from a notebook environment. While logging through a script, Neptune automatically stops tracking once the script has completed execution.

In [ ]:
run.stop()